In [1]:
import numpy as np
import torch as tr
from utils import *
from matplotlib import pyplot as plt

%reload_ext autoreload
%autoreload 2
%matplotlib inline

# training

In [ ]:
pr = 0.9
eplen = 50
switch = 25

TDupdate = False # 0:MC,1:TD

task = MRLBandit(eplen=eplen,banditpr=pr,switch_param=switch)
agent = ActorCritic(stsize=40,gamma=0.8,TDupdate=TDupdate)

neps = 25000
L = []
for ep in range(neps):
  expL = agent.unroll_ep(task)
  expD = unpack_expL(expL)
  agent.update(expD)
  ep_reward = expD['reward'].sum()
  L.append(ep_reward)


In [ ]:
R = np.array(L)
R = R.reshape(-1,20).mean(1)
plt.plot(R)

plt.axhline(eplen*pr,c='g')
plt.axhline(eplen/2,c='r')

# eval

In [ ]:
switch_ev = 25
eplen=100
pr=1
task_ev = MRLBandit(eplen=eplen,banditpr=pr,switch_param=switch_ev)

neps_ev = 1000
Rev = np.zeros([neps_ev,eplen+1])
for ep in range(neps_ev):
  expL = agent.unroll_ep(task)
  expD = unpack_expL(expL)
  Rev[ep] = expD['reward']

In [ ]:
M = Rev.mean(0)
S = Rev.std(0)/np.sqrt(len(Rev))
plt.plot(M)
plt.fill_between(range(len(M)),M-S,M+S,alpha=0.2)